In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('C:/Users/chanyoung/Desktop/Neural-GC-master/lorenz_96_10_10_1000.csv')
X = df[['a','b']].values

In [2]:
def data_split(X, timelag = 10):
    data = []
    Y = []
    for i in range(len(X) - 11):
        data.append(X[i:i+10])
        Y.append(X[i+11])

    return data, Y

In [17]:
class RBFtimenet2d(object):
    # This is one layer RBFnn, one-dims
    def __init__(self, timelag = 10, lr = 0.01, epochs = 100, inferStds = True, seed = 1234):
        self.lr = lr
        self.epochs = epochs
        self.inferStds = inferStds
        self.seed = seed
        self.timelag = timelag
        np.random.seed(self.seed)


    def cluster(self, X):
        # simple cluster means and stds list about time series data
        clusters = []
        stds = []
        cov = []
        for x in X:

            clu1, clu2 = np.mean(x[:, 0]), np.mean(x[:, 1])
            clusters.append([clu1, clu2])
            
            '''
            std1, std2 = np.std(x[:, 0]), np.std(x[:, 1])
            stds.append([std1, std2])
            '''
            cov.append(np.cov((x.T)))

        return clusters, cov
        # 확인 완료

    def rbf(self, x, clusters, cov):
        return np.exp(-1*((x[0] -clusters[0])**2 + (x[1] - clusters[1])**2)/
                      (np.linalg.inv(cov) * np.linalg.inv(cov)))

    def fit(self, X, y):

        self.c, self.cov = self.cluster(X)
        self.w_xx = np.random.randn(len(X),10)
        self.w_yy = np.random.randn(len(X), 10)
        self.b= np.random.randn(len(X),1)


        # training
        loss_list = []
        loss_mean_list = []
        F_list_epoch = []
        for epoch in range(self.epochs):
            loss_list2 = []
            print('{} epoch train'.format(epoch))
            pred_list = []
            for i in range(len(X)):
                rbf_X = np.array([self.rbf(x, self.c[i], self.cov[i]) for x in X[i]])
                
                rbf_xx = np.array([rbf_X[i][0][0] for i in range(len(rbf_X))])
                rbf_yy = np.array([rbf_X[k][1][1] for k in range(len(rbf_X))])
                
                F = self.w_xx[i].T.dot(rbf_xx) + self.w_yy[i].T.dot(rbf_yy) + self.b[i]
                
                loss = -(y[i][1] - F)
                
                # loss predict value save
                loss_list2.append(np.abs(loss))
                pred_list.append(F)

                # weight, bias, center, sigma update
                self.w_xx[i] += self.lr * rbf_xx * loss
                self.w_yy[i] += self.lr * rbf_yy * loss
                self.b[i] += self.lr * loss
                
                self.c[i] += self.lr * loss
                
            F_list_epoch.append(pred_list)
            loss_list.append(loss_list2)
            loss_mean_list.append(np.mean(loss_list2))
            print("{} epoch loss:".format(epoch), np.mean(loss_list2))
            print('---------------------------------------')
            print()

            if epoch >= 5 and (loss_mean_list[epoch] > min(loss_mean_list[epoch - 5:epoch - 1])):
                print("early stopping at {} epoch".format(epoch))
                return loss_mean_list, F_list_epoch, loss_list

            else:
                continue

        return loss_mean_list, F_list_epoch, loss_list

In [18]:
df = pd.read_csv('C:/Users/chanyoung/Desktop/Neural-GC-master/lorenz_96_10_10_1000.csv')
X2d = df[['a','b']].values

lag = 10
X_2d, Y = data_split(X2d,timelag = lag)
rbfnet2d = RBFtimenet2d(timelag = lag,lr=1e-3,epochs = 100)
loss_mean_list, F_list_epoch, loss_list = rbfnet2d.fit(X_2d, Y)

0 epoch train
0 epoch loss: 4.313919015707323
---------------------------------------

1 epoch train
1 epoch loss: 4.319882511289515
---------------------------------------

2 epoch train
2 epoch loss: 4.32587904326755
---------------------------------------

3 epoch train
3 epoch loss: 4.331910655652857
---------------------------------------

4 epoch train
4 epoch loss: 4.337979090949457
---------------------------------------

5 epoch train
5 epoch loss: 4.344085727856526
---------------------------------------

early stopping at 5 epoch


In [11]:
a = [[2,2], [3,3]]

In [14]:
 np.array([x for i in range(len(a))])

NameError: name 'i' is not defined